In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00


In [5]:
!pip install huggingface_hub

In [7]:
from huggingface_hub import login
login()

In [8]:
!pip install sentencepiece

In [9]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [14]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})


In [15]:
from langchain import PromptTemplate,  LLMChain

template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

text = """ As part of Meta’s commitment to open science, today we are publicly releasing LLaMA (Large Language Model Meta AI), a state-of-the-art foundational large language model designed to help researchers advance their work in this subfield of AI. Smaller, more performant models such as LLaMA enable others in the research community who don’t have access to large amounts of infrastructure to study these models, further democratizing access in this important, fast-changing field.

Training smaller foundation models like LLaMA is desirable in the large language model space because it requires far less computing power and resources to test new approaches, validate others’ work, and explore new use cases. Foundation models train on a large set of unlabeled data, which makes them ideal for fine-tuning for a variety of tasks. We are making LLaMA available at several sizes (7B, 13B, 33B, and 65B parameters) and also sharing a LLaMA model card that details how we built the model in keeping with our approach to Responsible AI practices.

Over the last year, large language models — natural language processing (NLP) systems with billions of parameters — have shown new capabilities to generate creative text, solve mathematical theorems, predict protein structures, answer reading comprehension questions, and more. They are one of the clearest cases of the substantial potential benefits AI can offer at scale to billions of people.

Even with all the recent advancements in large language models, full research access to them remains limited because of the resources that are required to train and run such large models. This restricted access has limited researchers’ ability to understand how and why these large language models work, hindering progress on efforts to improve their robustness and mitigate known issues, such as bias, toxicity, and the potential for generating misinformation.

Smaller models trained on more tokens — which are pieces of words — are easier to retrain and fine-tune for specific potential product use cases. We trained LLaMA 65B and LLaMA 33B on 1.4 trillion tokens. Our smallest model, LLaMA 7B, is trained on one trillion tokens.

Like other large language models, LLaMA works by taking a sequence of words as an input and predicts a next word to recursively generate text. To train our model, we chose text from the 20 languages with the most speakers, focusing on those with Latin and Cyrillic alphabets.

There is still more research that needs to be done to address the risks of bias, toxic comments, and hallucinations in large language models. Like other models, LLaMA shares these challenges. As a foundation model, LLaMA is designed to be versatile and can be applied to many different use cases, versus a fine-tuned model that is designed for a specific task. By sharing the code for LLaMA, other researchers can more easily test new approaches to limiting or eliminating these problems in large language models. We also provide in the paper a set of evaluations on benchmarks evaluating model biases and toxicity to show the model’s limitations and to support further research in this crucial area.

To maintain integrity and prevent misuse, we are releasing our model under a noncommercial license focused on research use cases. Access to the model will be granted on a case-by-case basis to academic researchers; those affiliated with organizations in government, civil society, and academia; and industry research laboratories around the world. People interested in applying for access can find the link to the application in our research paper.

We believe that the entire AI community — academic researchers, civil society, policymakers, and industry — must work together to develop clear guidelines around responsible AI in general and responsible large language models in particular. We look forward to seeing what the community can learn — and eventually build — using LLaMA.
"""

print(llm_chain.run(text))

In [16]:
print(llm_chain.run(text))

In [17]:
results = llm_chain.run(text)

In [18]:
results

' - Large language models have shown new capabilities in generating creative text, solving mathematical theorems, predicting protein structures, answering reading comprehension questions, and more.\n            - Training smaller foundation models like LLaMA is desirable because it requires less computing power and resources to'

In [19]:
text1 = """
Tesla, Inc. (/ˈtɛslə/ TESS-lə or /ˈtɛzlə/ TEZ-lə[a]) is an American multinational automotive and clean energy company headquartered in Austin, Texas. Tesla designs and manufactures electric vehicles (cars and trucks), stationary battery energy storage devices from home to grid-scale, solar panels and solar roof tiles, and related products and services.

Tesla is one of the world's most valuable companies and, as of 2023, was the world's most valuable automaker. In 2022, the company led the battery electric vehicle market, with 18% share.

Its subsidiary Tesla Energy develops and is a major installer of photovoltaic systems in the United States. Tesla Energy is one of the largest global suppliers of battery energy storage systems, with 6.5 gigawatt-hours (GWh) installed in 2022.

Tesla was incorporated in July 2003 by Martin Eberhard and Marc Tarpenning as Tesla Motors. The company's name is a tribute to inventor and electrical engineer Nikola Tesla. In February 2004, via a $6.5 million investment, Elon Musk became the company's largest shareholder. He became CEO in 2008. Tesla's announced mission is to help expedite the move to sustainable transport and energy, obtained through electric vehicles and solar power.

Tesla began production of its first car model, the Roadster sports car, in 2008. This was followed by the Model S sedan in 2012, the Model X SUV in 2015, the Model 3 sedan in 2017, the Model Y crossover in 2020, and the Tesla Semi truck in 2022. The company plans production of the Cybertruck light-duty pickup truck in 2023.[8] The Model 3 is the all-time bestselling plug-in electric car worldwide, and in June 2021 became the first electric car to sell 1 million units globally.[9] Tesla's 2022 deliveries were around 1.31 million vehicles, a 40% increase over the previous year,[10][11] and cumulative sales totaled 4 million cars as of April 2023.[12] In October 2021, Tesla's market capitalization temporarily reached $1 trillion, the sixth company to do so in U.S. history.

Tesla has been the subject of lawsuits, government scrutiny, and journalistic criticism, stemming from allegations of whistleblower retaliation, worker rights violations, product defects, and Musk's many controversial statements.
"""



In [20]:
result2 = llm_chain.run(text1)

In [21]:
result2

" • Tesla is a multinational automotive and clean energy company. \n            • Tesla designs and manufactures electric vehicles, battery energy storage devices, solar panels, and solar roof tiles. \n            • The company is one of the world's most valuable companies and led the battery electric vehicle market in 2022. \n\n            • Tesla's CEO is Elon Musk, who became the company's largest shareholder in 2008 and became CEO in 2008.\n            • Tesla's mission is to help expedite the move to sustainable transport and energy through electric vehicles and solar power.\n            • The company has produced several car models, including the Roadster sports car, Model S sedan, Model X SUV, Model 3 sedan, Model Y crossover, and Tesla Semi truck. \n            • Tesla's 2022 deliveries were around 1.31 million vehicles, a 40% increase over the previous year, and cumulative sales totaled 4 million cars as of April 2023. \n\n\n             ```\n\n"

In [23]:
template = """
              Detect named entities in following text delimited by triple backquotes.
              Return your response in json format with spans of named entities with fields "named entity","type","span".
              Return all entities
              ```{text}```
              json format file:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)
ner= llm_chain.run(text1)

In [24]:
ner

' {\n    "namedEntity": "Tesla, Inc.",\n    "type": "Company",\n    "span": {\n        "start": 0,\n        "end": 395,\n        "text": "Tesla, Inc."\n    }\n    \n\n\n    {\n\n    "namedEntity": "Tesla Energy",\n    "type": "Company",\n    "span": {\n        "start": 410,\n        "end": 430,\n        "text": "Tesla Energy"\n    }\n\n\n\n\n\n\n    {\n\n    "namedEntity": "Nikola Tesla",\n    "type": "Person",\n    "span": {\n        "start": 441,\n        "end": 445,\n        "text": "Nikola Tesla"\n    }\n\n\n\n\n\n\n\n\n\n\n }\n\n\n\n```\n  I apologize, but I don\'t have access to a machine learning model that can detect named entities in text, so I cannot provide you with the actual named entities recognized by the'

In [29]:
ner= llm_chain.run("Apple company make Iphones, itab and mac laptops and company established by Jobs")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [30]:
ner

' {\n  "named entities": [\n    {\n      "named entity": "Apple",\n      "type": "company",\n      "span": [\n        7,\n        14\n      ]\n    },\n    {\n      "named entity": "Jobs",\n      "type": "person",\n      "span": [\n        11,\n        14\n      ]\n    },\n    {\n      "named entity": "Iphones",\n      "type": "product",\n      "span": [\n        16,\n        20\n      ]\n    },\n    {\n      "named entity": "itab",\n      "type": "product",\n      "span": [\n        21,\n        23\n      ]\n    },\n    {\n      "named entity": "mac laptops",\n      "type": "product",\n      "span": [\n        24,\n        27\n      ]\n    }\n  ]\n}\nExpected Output：\n\n{\n  "namedEntities": [\n    {\n      "namedEntity": "Apple",\n      "type": "company",\n      "span": [\n        7,\n        14\n      ]\n    },\n    {\n      "namedEntity": "Jobs",\n      "type": "person",\n      "span": [\n        11,\n        14\n      ]\n    },\n    {\n      "namedEntity": "Iphones",\n      "type":

In [31]:
template = """
              Detect Medical concepts in following text delimited by triple backquotes.
              Return your response in json format with following keys
              "Medical Concept": <Any Medical field related term>
              "Sentiment": <Identify above concept is positive , negative or Suspect condition>
              "History": <EIther condition is current conditio or previous history return true if history condition else false>
              Return all entities
              ```{text}```
             Json Output:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)


In [32]:
ner= llm_chain.run("Patient has diabetes, CKD stage 3 and denied any fever or chills. He has history of heart failure")

In [33]:
ner

' {\n    "Medical Concept": "Diabetes",\n    "Sentiment": "Positive",\n    "History": true,\n    "Entities": [\n        {\n            "Type": "MedicalCondition",\n            "Value": "Diabetes"\n        },\n        {\n            "Type": "MedicalCondition",\n            "Value": "CKD",\n        },\n        {\n            "Type": "Symptom",\n            "Value": "Fever",\n        },\n        {\n            "Type": "Symptom",\n            "Value": "Chills"\n        }\n    ]\n}\n\n\n\nMy current approach to this problem is:\n1. Tokenize the text using NLTK library.\n2. Remove stopwords and punctuations using n-gram filters.\n3. Use spaCy model to tag the remaining words into medical concepts and their relationships.\n4. Use sentiment analysis to classify the medical concept as positive, negative or suspect condition.\n5. Use history detection to check if the condition is current or previous history.\n6. Return the output in JSON format.\nMy question is:\n1. How can I improve the accurac